# Backtesting ML Regression-Based

In [ ]:
???

Classification Strategy | Regression Strategy
-|-
![](src/res_classification.png) | ![](src/res_regression.png)

## Load the model

In [6]:
import pickle

In [7]:
with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

/Users/sota/miniforge3/envs/backtesting/lib/python3.11/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [9]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,56.325228,56.582507,55.902560,56.058762,21220800,1.549143,UP
2016-12-09,56.214968,56.959234,56.169027,56.940857,27349400,0.321692,UP
...,...,...,...,...,...,...,...
2023-03-14,256.750000,261.070007,255.860001,260.790009,33620300,1.751806,UP
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000,3.895731,UP


## Backtesting.py Library

### Create your Strategy Class

In [10]:
from backtesting import Backtest, Strategy

In [11]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Define initial conditions

In [12]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [13]:
df_explanatory.iloc[-1:, :]

,Open,High,Low,Close,Volume
Date,,,,,
2023-03-15,259.980011,266.480011,259.209991,265.440002,46028000


In [14]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting

In [15]:
results = bt.run()

### Interpret backtesting results

In [ ]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2023-03-15 00:00:00
Duration,2288 days 00:00:00
Exposure Time [%],91.941624
Equity Final [$],54828.785476
Equity Peak [$],61047.729861
Return [%],448.287855


### Visualize the strategy performance

In [ ]:
bt.plot(filename='reports_backtesting/backtesting_regression.html')

Row(id='1408', ...)